# Equation Solver
In this project we came up with multiple methods to solve handwritten equations and provide a live demo of the process.

### Project Repo 
https://github.com/opencampus-sh/project_template_folder

## Participants:

- Jeremy von Winceklmann
- Fynn Junge
- Tim Staubert

### Course and Semester
Deep Leraning WS 2022/2023

In [23]:
import torch
from PIL import Image
import numpy as np
from torch import nn as nn

In [25]:
class CNN(nn.Module):                  #Score: 99.2%
    def __init__(self, out_classes):
        super(CNN, self).__init__()

        # Layer #1
        self.conv1 = nn.Sequential(     #Input 28*28*1     
            nn.Conv2d(
                in_channels=1,              
                out_channels=32,            
                kernel_size=5,              
                stride=1,                   
                padding=0,                  
            ),                          #Output 24*24*32                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2) #Output 12*12*32
        )
        # Layer #2
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=32,              
                out_channels=64,            
                kernel_size=3,              
                stride=1,                   
                padding=0,
            ),                          #Output 10*10*64
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2), #Output 5*5*64              
        )
        self.dense = nn.Sequential(
            nn.Linear(5*5*64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, out_classes)
        )
    def forward(self, x):
        # Put through 2 convolution layers and a 3L deep network with dropouts
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  #Flatten
        output = self.dense(x)
        return output

In [30]:
clf = CNN(12)
clf.load_state_dict(torch.load('../resources/cnn2.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [42]:
img = Image.open("-_18.jpg")

In [43]:
img

In [45]:
def predict(two_d_array):
    four_d_array = test = np.expand_dims(two_d_array,axis=(0,1))
    clf.eval()
    output = clf(torch.Tensor(test))
    _, predicted = torch.max(output.data, 1)
    classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "-"]
    return classes[predicted]   

In [46]:
predict(np.asarray(img))

'-'